In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras import layers


Num GPUs Available:  1


In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In [3]:
DATASET_PATH = '/home/datasets/Project_data/'

In [4]:
class DataGenerator():
    def __init__(self):
        'Initialization'
        self.train_doc = np.random.permutation(open(DATASET_PATH + 'train.csv').readlines())
        self.val_doc = np.random.permutation(open(DATASET_PATH + 'val.csv').readlines())
        
        self.train_path = DATASET_PATH + 'train'
        self.val_path = DATASET_PATH + 'val'
        
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
        
        self.img_idx = [0,2,4,6,8,10,12,14,16,18,20,22,24,25,26,27,28,29]
    
    def initialize_parameters(self,batch_size=10,img_height=100,img_width=100,num_epochs=5,
                              n_channels=3,ablation=None,mode='train'):
        self.batch_size = batch_size
        self.num_epochs = num_epochs
        self.img_height = img_height
        self.img_width = img_width
        self.channels = n_channels
        self.num_classes = 5
        self.frames = 30
    
    def generator(self,source_path, folder_list, transform=False):
        batch_size=self.batch_size
        
        while True:
            t = np.random.permutation(folder_list)
            num_batches = np.floor(len(folder_list)/self.batch_size).astype(int)
            
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data, batch_labels = self.fetch_batch(source_path,t,batch,self.batch_size,self.img_idx,transform)
                yield batch_data, batch_labels
            
            pending_batches = (len(folder_list) - num_batches*self.batch_size)
            
            if pending_batches > 0:
                batch_data, batch_labels = self.fetch_batch(source_path,t,batch,pending_batches,self.img_idx,transform)
                yield batch_data, batch_labels
    
    def fetch_batch(self,source_path,t,batch,batch_size,img_idx,transform):
        # x is the number of images you use for each video, 
        # (y,z) is the final size of the input images and 3 is the number of channels RGB
        batch_data = np.zeros((batch_size,len(img_idx),self.img_height,self.img_width,3))
        batch_labels = np.zeros((batch_size,5))
        
        if (transform):
            batch_data_affine = np.zeros((batch_size,len(img_idx),self.img_height,self.img_width,3))

        for folder in range(batch_size):
            # read all the images in the folder
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) 
            
            img_idx = self.img_idx
            
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                if image.shape[0] != image.shape[1]:
                    image = image[0:120, 10:150]
                
                image = cv2.resize(image, (self.img_height,self.img_width), interpolation = cv2.INTER_AREA)

                image = image/255
                
                batch_data[folder,idx,:,:,0] = image[:,:,0]
                batch_data[folder,idx,:,:,1] = image[:,:,1]
                batch_data[folder,idx,:,:,2] = image[:,:,2]
                
                #Affine transformation to shift image up/down or right/left
                if (transform):
                    height, width = image.shape[:2]
                    tx, ty = width / 4, height / 4
                    tx = np.random.randint(-tx,tx)
                    ty = np.random.randint(-ty,ty)
                    translation_matrix = np.array([[1, 0, tx],
                                                   [0, 1, ty]], dtype=np.float32)

                    translated_image = cv2.warpAffine(src=image, M=translation_matrix, dsize=(width, height))

                    batch_data_affine[folder,idx,:,:,0] = translated_image[:,:,0]
                    batch_data_affine[folder,idx,:,:,1] = translated_image[:,:,1]
                    batch_data_affine[folder,idx,:,:,2] = translated_image[:,:,2]

            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1

        if (transform):
            batch_data=np.concatenate([batch_data,batch_data_affine])
            batch_labels=np.concatenate([batch_labels,batch_labels])
        
        return (batch_data, batch_labels)
    
    def train_model(self,model,transform=False):
        train_generator = self.generator(self.train_path, self.train_doc,transform)
        val_generator = self.generator(self.val_path, self.val_doc,transform)
        
        curr_dt_time = datetime.datetime.now()
        
        model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
        if not os.path.exists(model_name):
            os.mkdir(model_name)

        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
        
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto',save_freq = 'epoch')
        
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1) # write the REducelronplateau code here
        
        earlystop = EarlyStopping(monitor="val_loss", min_delta=0,patience=10,verbose=1)
        
        callbacks_list = [checkpoint, LR, earlystop]
        
        if (self.num_train_sequences%self.batch_size) == 0:
            #steps_per_epoch = np.floor(num_train_sequences/batch_size).astype(int)
            steps_per_epoch = int(self.num_train_sequences/self.batch_size)
        else:
            #steps_per_epoch = (num_train_sequences//batch_size) + 1
            steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1

        if (self.num_val_sequences%self.batch_size) == 0:
            #validation_steps = np.floor(num_val_sequences/batch_size).astype(int)
            validation_steps = int(self.num_val_sequences/self.batch_size)
        else:
            #validation_steps = (num_val_sequences//batch_size) + 1
            validation_steps = (self.num_val_sequences//self.batch_size) + 1
        
        
        history = model.fit(train_generator, 
                            steps_per_epoch=steps_per_epoch, 
                            epochs=self.num_epochs, 
                            verbose=1, 
                            callbacks=callbacks_list, 
                            validation_data=val_generator, 
                            validation_steps=validation_steps, 
                            class_weight=None, workers=1, initial_epoch=0)
        
        return history
    
    def define_model(self):
        pass

In [16]:
class Conv3DModel1(DataGenerator):

    def model_structure(self):
        #write your model here
        #normalization_layer = layers.experimental.preprocessing.Rescaling(1./255, 
        #input_shape=(img_frames,img_height, img_width, 3))

        #model3d = Sequential([normalization_layer])
        model = Sequential()

        model.add(layers.Conv3D(16,(3,3,3), activation="relu",data_format='channels_first', 
                                input_shape=(len(self.img_idx),self.img_height, self.img_width, 3)))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        model.add(layers.Conv3D(32,(3,3,3), activation="relu",))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(64,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(128,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        model.add(layers.Flatten())

        #model.add(layers.Dense(128, activation="relu"))
        #model.add(BatchNormalization())
        #model.add(layers.Dropout(0.5))

        model.add(layers.Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))

        model.add(layers.Dense(5,activation='softmax'))

        optimiser = optimizers.SGD()
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [17]:
conv_3d1=Conv3DModel1()
conv_3d1.initialize_parameters(batch_size=10,num_epochs=5,img_height=150,img_width=150)
conv_3d1_model=conv_3d1.model_structure()
conv_3d1_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization_3 (Batch (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 124416)           

In [7]:
conv_3d1.train_model(model=conv_3d1_model)

Epoch 1/5
67/67 [==============================] - ETA: 0s - loss: 1.6690 - categorical_accuracy: 0.2790
Epoch 00001: val_loss improved from inf to 1.61919, saving model to model_init_2021-06-2320_31_32.063034/model-00001-1.66903-0.27903-1.61919-0.21000.h5
67/67 [==============================] - 40s 596ms/step - loss: 1.6690 - categorical_accuracy: 0.2790 - val_loss: 1.6192 - val_categorical_accuracy: 0.2100
Epoch 2/5
67/67 [==============================] - ETA: 0s - loss: 1.4339 - categorical_accuracy: 0.3725
Epoch 00002: val_loss did not improve from 1.61919
67/67 [==============================] - 40s 593ms/step - loss: 1.4339 - categorical_accuracy: 0.3725 - val_loss: 1.9499 - val_categorical_accuracy: 0.2800
Epoch 3/5
67/67 [==============================] - ETA: 0s - loss: 1.4209 - categorical_accuracy: 0.3982
Epoch 00003: val_loss did not improve from 1.61919
67/67 [==============================] - 39s 586ms/step - loss: 1.4209 - categorical_accuracy: 0.3982 - val_loss: 1.668

In [25]:
class Conv3DModel2(DataGenerator):

    def model_structure(self):
        #write your model here
        #normalization_layer = layers.experimental.preprocessing.Rescaling(1./255, 
        #input_shape=(img_frames,img_height, img_width, 3))

        #model3d = Sequential([normalization_layer])
        model = Sequential()

        model.add(layers.Conv3D(16,(3,3,3), activation="relu",data_format='channels_first', 
                                input_shape=(len(self.img_idx),self.img_height, self.img_width, 3)))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        model.add(layers.Conv3D(32,(3,3,3), activation="relu",))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(64,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(128,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        model.add(layers.Flatten())

        #model.add(layers.Dense(128, activation="relu"))
        #model.add(BatchNormalization())
        #model.add(layers.Dropout(0.5))

        model.add(layers.Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))

        model.add(layers.Dense(5,activation='softmax'))

        optimiser = optimizers.SGD(lr=0.001)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [26]:
conv_3d2=Conv3DModel2()
conv_3d2.initialize_parameters(batch_size=10,num_epochs=5,img_height=150,img_width=150)
conv_3d2_model=conv_3d2.model_structure()
conv_3d2_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_6 (Conv3D)            (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization_9 (Batch (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_10 (Batc (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 124416)           

In [27]:
conv_3d2.train_model(model=conv_3d2_model)

Epoch 1/5
67/67 [==============================] - ETA: 0s - loss: 1.6390 - categorical_accuracy: 0.3560
Epoch 00001: val_loss improved from inf to 2.21542, saving model to model_init_2021-06-2320_59_01.439695/model-00001-1.63895-0.35596-2.21542-0.21000.h5
67/67 [==============================] - 39s 587ms/step - loss: 1.6390 - categorical_accuracy: 0.3560 - val_loss: 2.2154 - val_categorical_accuracy: 0.2100
Epoch 2/5
67/67 [==============================] - ETA: 0s - loss: 1.2922 - categorical_accuracy: 0.4555
Epoch 00002: val_loss did not improve from 2.21542
67/67 [==============================] - 39s 587ms/step - loss: 1.2922 - categorical_accuracy: 0.4555 - val_loss: 3.2276 - val_categorical_accuracy: 0.2400
Epoch 3/5
67/67 [==============================] - ETA: 0s - loss: 1.0856 - categorical_accuracy: 0.5822
Epoch 00003: val_loss did not improve from 2.21542
67/67 [==============================] - 39s 583ms/step - loss: 1.0856 - categorical_accuracy: 0.5822 - val_loss: 3.604

In [28]:
conv_3d2=Conv3DModel2()
conv_3d2.initialize_parameters(batch_size=10,num_epochs=10,img_height=150,img_width=150)
conv_3d2_model=conv_3d2.model_structure()
conv_3d2_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization_12 (Batc (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_13 (Batc (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 124416)           

In [29]:
conv_3d2.train_model(model=conv_3d2_model)

Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.7122 - categorical_accuracy: 0.3318
Epoch 00001: val_loss improved from inf to 1.72030, saving model to model_init_2021-06-2321_04_11.241700/model-00001-1.71219-0.33183-1.72030-0.21000.h5
67/67 [==============================] - 40s 590ms/step - loss: 1.7122 - categorical_accuracy: 0.3318 - val_loss: 1.7203 - val_categorical_accuracy: 0.2100
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.3428 - categorical_accuracy: 0.4555
Epoch 00002: val_loss did not improve from 1.72030
67/67 [==============================] - 39s 586ms/step - loss: 1.3428 - categorical_accuracy: 0.4555 - val_loss: 2.8117 - val_categorical_accuracy: 0.2100
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.1191 - categorical_accuracy: 0.5475
Epoch 00003: val_loss did not improve from 1.72030
67/67 [==============================] - 39s 583ms/step - loss: 1.1191 - categorical_accuracy: 0.5475 - val_loss: 2.

In [30]:
class Conv3DModel3(DataGenerator):

    def model_structure(self):
        #write your model here
        #normalization_layer = layers.experimental.preprocessing.Rescaling(1./255, 
        #input_shape=(img_frames,img_height, img_width, 3))

        #model3d = Sequential([normalization_layer])
        model = Sequential()

        model.add(layers.Conv3D(16,(3,3,3), activation="relu",data_format='channels_first', 
                                input_shape=(len(self.img_idx),self.img_height, self.img_width, 3)))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        model.add(layers.Conv3D(32,(3,3,3), activation="relu",))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(64,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(128,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        model.add(layers.Flatten())

        #model.add(layers.Dense(128, activation="relu"))
        #model.add(BatchNormalization())
        #model.add(layers.Dropout(0.5))

        model.add(layers.Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))

        model.add(layers.Dense(5,activation='softmax'))

        optimiser = optimizers.SGD(lr=0.0001)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [32]:
conv_3d3=Conv3DModel3()
conv_3d3.initialize_parameters(batch_size=10,num_epochs=15,img_height=150,img_width=150)
conv_3d3_model=conv_3d3.model_structure()
conv_3d3_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_12 (Conv3D)           (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization_18 (Batc (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_19 (Batc (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 124416)           

In [33]:
conv_3d3.train_model(model=conv_3d3_model)

Epoch 1/15
67/67 [==============================] - ETA: 0s - loss: 1.8115 - categorical_accuracy: 0.2745
Epoch 00001: val_loss improved from inf to 2.19505, saving model to model_init_2021-06-2321_14_49.604082/model-00001-1.81152-0.27451-2.19505-0.16000.h5
67/67 [==============================] - 40s 601ms/step - loss: 1.8115 - categorical_accuracy: 0.2745 - val_loss: 2.1950 - val_categorical_accuracy: 0.1600
Epoch 2/15
67/67 [==============================] - ETA: 0s - loss: 1.5128 - categorical_accuracy: 0.3846
Epoch 00002: val_loss did not improve from 2.19505
67/67 [==============================] - 39s 589ms/step - loss: 1.5128 - categorical_accuracy: 0.3846 - val_loss: 2.7469 - val_categorical_accuracy: 0.1500
Epoch 3/15
67/67 [==============================] - ETA: 0s - loss: 1.3771 - categorical_accuracy: 0.4404
Epoch 00003: val_loss did not improve from 2.19505
67/67 [==============================] - 39s 584ms/step - loss: 1.3771 - categorical_accuracy: 0.4404 - val_loss: 2.

In [34]:
conv_3d4=Conv3DModel3()
conv_3d4.initialize_parameters(batch_size=20,num_epochs=15,img_height=150,img_width=150)
conv_3d4_model=conv_3d4.model_structure()
conv_3d4_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_14 (Conv3D)           (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization_21 (Batc (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d_14 (MaxPooling (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_22 (Batc (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 124416)           

In [35]:
conv_3d4.train_model(model=conv_3d4_model)

Epoch 1/15
33/34 [============================>.] - ETA: 0s - loss: 2.0389 - categorical_accuracy: 0.2970
Epoch 00001: val_loss improved from inf to 1.59088, saving model to model_init_2021-06-2321_26_51.045085/model-00001-2.04375-0.29563-1.59088-0.24000.h5
34/34 [==============================] - 39s 1s/step - loss: 2.0437 - categorical_accuracy: 0.2956 - val_loss: 1.5909 - val_categorical_accuracy: 0.2400
Epoch 2/15
34/34 [==============================] - ETA: 0s - loss: 1.5787 - categorical_accuracy: 0.3756
Epoch 00002: val_loss did not improve from 1.59088
34/34 [==============================] - 38s 1s/step - loss: 1.5787 - categorical_accuracy: 0.3756 - val_loss: 1.6567 - val_categorical_accuracy: 0.3100
Epoch 3/15
34/34 [==============================] - ETA: 0s - loss: 1.3593 - categorical_accuracy: 0.4751
Epoch 00003: val_loss improved from 1.59088 to 1.52043, saving model to model_init_2021-06-2321_26_51.045085/model-00003-1.35926-0.47511-1.52043-0.27000.h5
34/34 [==========

In [40]:
conv_3d5=Conv3DModel3()
conv_3d5.initialize_parameters(batch_size=20,num_epochs=25,img_height=150,img_width=150)
conv_3d5_model=conv_3d5.model_structure()
conv_3d5_model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_18 (Conv3D)           (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization_27 (Batc (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d_18 (MaxPooling (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_28 (Batc (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 124416)           

In [41]:
conv_3d5.train_model(model=conv_3d5_model)

Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 2.0028 - categorical_accuracy: 0.2851
Epoch 00001: val_loss improved from inf to 1.76088, saving model to model_init_2021-06-2321_54_27.730621/model-00001-2.00279-0.28507-1.76088-0.16000.h5
34/34 [==============================] - 38s 1s/step - loss: 2.0028 - categorical_accuracy: 0.2851 - val_loss: 1.7609 - val_categorical_accuracy: 0.1600
Epoch 2/25
34/34 [==============================] - ETA: 0s - loss: 1.5251 - categorical_accuracy: 0.4163
Epoch 00002: val_loss did not improve from 1.76088
34/34 [==============================] - 38s 1s/step - loss: 1.5251 - categorical_accuracy: 0.4163 - val_loss: 2.2840 - val_categorical_accuracy: 0.1500
Epoch 3/25
34/34 [==============================] - ETA: 0s - loss: 1.3102 - categorical_accuracy: 0.5038
Epoch 00003: val_loss did not improve from 1.76088
34/34 [==============================] - 38s 1s/step - loss: 1.3102 - categorical_accuracy: 0.5038 - val_loss: 2.7172 - va

In [5]:
class Conv3DModel4(DataGenerator):

    def model_structure(self):
        #write your model here
        #normalization_layer = layers.experimental.preprocessing.Rescaling(1./255, 
        #input_shape=(img_frames,img_height, img_width, 3))

        #model3d = Sequential([normalization_layer])
        model = Sequential()

        model.add(layers.Conv3D(16,(3,3,3), activation="relu",data_format='channels_first', 
                                input_shape=(len(self.img_idx),self.img_height, self.img_width, 3)))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        model.add(layers.Conv3D(32,(3,3,3), activation="relu",))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(64,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(128,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        model.add(layers.Flatten())

        #model.add(layers.Dense(128, activation="relu"))
        #model.add(BatchNormalization())
        #model.add(layers.Dropout(0.5))

        model.add(layers.Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))

        model.add(layers.Dense(5,activation='softmax'))

        optimiser = optimizers.SGD(lr=0.001)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [6]:
conv_3d6=Conv3DModel4()
conv_3d6.initialize_parameters(batch_size=20,num_epochs=25,img_height=150,img_width=150)
conv_3d6_model=conv_3d6.model_structure()
conv_3d6_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten (Flatten)            (None, 124416)            0

In [7]:
conv_3d6.train_model(model=conv_3d6_model)

Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 1.7036 - categorical_accuracy: 0.2836
Epoch 00001: val_loss improved from inf to 1.79321, saving model to model_init_2021-06-2400_30_19.348045/model-00001-1.70360-0.28356-1.79321-0.24000.h5
34/34 [==============================] - 39s 1s/step - loss: 1.7036 - categorical_accuracy: 0.2836 - val_loss: 1.7932 - val_categorical_accuracy: 0.2400
Epoch 2/25
34/34 [==============================] - ETA: 0s - loss: 1.4079 - categorical_accuracy: 0.4133
Epoch 00002: val_loss did not improve from 1.79321
34/34 [==============================] - 38s 1s/step - loss: 1.4079 - categorical_accuracy: 0.4133 - val_loss: 2.0428 - val_categorical_accuracy: 0.2500
Epoch 3/25
34/34 [==============================] - ETA: 0s - loss: 1.2924 - categorical_accuracy: 0.4721
Epoch 00003: val_loss did not improve from 1.79321
34/34 [==============================] - 38s 1s/step - loss: 1.2924 - categorical_accuracy: 0.4721 - val_loss: 2.2337 - va

In [8]:
conv_3d7=Conv3DModel4()
conv_3d7.initialize_parameters(batch_size=25,num_epochs=25,img_height=150,img_width=150)
conv_3d7_model=conv_3d7.model_structure()
conv_3d7_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_2 (Conv3D)            (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization_3 (Batch (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 124416)           

In [9]:
conv_3d7.train_model(model=conv_3d7_model)

Epoch 1/25
27/27 [==============================] - ETA: 0s - loss: 1.5879 - categorical_accuracy: 0.3967
Epoch 00001: val_loss improved from inf to 1.63169, saving model to model_init_2021-06-2400_47_10.093635/model-00001-1.58789-0.39668-1.63169-0.19000.h5
27/27 [==============================] - 38s 1s/step - loss: 1.5879 - categorical_accuracy: 0.3967 - val_loss: 1.6317 - val_categorical_accuracy: 0.1900
Epoch 2/25
27/27 [==============================] - ETA: 0s - loss: 1.0064 - categorical_accuracy: 0.6199
Epoch 00002: val_loss did not improve from 1.63169
27/27 [==============================] - 38s 1s/step - loss: 1.0064 - categorical_accuracy: 0.6199 - val_loss: 2.2261 - val_categorical_accuracy: 0.1800
Epoch 3/25
27/27 [==============================] - ETA: 0s - loss: 0.8544 - categorical_accuracy: 0.6772
Epoch 00003: val_loss did not improve from 1.63169
27/27 [==============================] - 37s 1s/step - loss: 0.8544 - categorical_accuracy: 0.6772 - val_loss: 2.3544 - va

In [5]:
class Conv3DModel5(DataGenerator):

    def model_structure(self):
        #write your model here
        #normalization_layer = layers.experimental.preprocessing.Rescaling(1./255, 
        #input_shape=(img_frames,img_height, img_width, 3))

        #model3d = Sequential([normalization_layer])
        model = Sequential()

        model.add(layers.Conv3D(16,(3,3,3), activation="relu",data_format='channels_first', 
                                input_shape=(len(self.img_idx),self.img_height, self.img_width, 3)))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        model.add(layers.Conv3D(32,(3,3,3), activation="relu",))
        model.add(BatchNormalization())
        model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(64,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        #model.add(layers.Conv3D(128,(3,3,3), activation="relu",))
        #model.add(BatchNormalization())
        #model.add(layers.MaxPool3D())

        model.add(layers.Flatten())

        #model.add(layers.Dense(128, activation="relu"))
        #model.add(BatchNormalization())
        #model.add(layers.Dropout(0.5))

        model.add(layers.Dense(32, activation="relu"))
        model.add(BatchNormalization())
        model.add(layers.Dropout(0.5))

        model.add(layers.Dense(5,activation='softmax'))

        optimiser = optimizers.Adam(lr=0.001)
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        return model

In [6]:
conv_3d8=Conv3DModel5()
conv_3d8.initialize_parameters(batch_size=20,num_epochs=25,img_height=150,img_width=150)
conv_3d8_model=conv_3d8.model_structure()
conv_3d8_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 16, 148, 148, 1)   7792      
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 148, 148, 1)   4         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 8, 74, 74, 1)      0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 6, 72, 72, 32)     896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 6, 72, 72, 32)     128       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 3, 36, 36, 32)     0         
_________________________________________________________________
flatten (Flatten)            (None, 124416)            0

In [7]:
conv_3d8.train_model(model=conv_3d8_model)

Epoch 1/25
34/34 [==============================] - ETA: 0s - loss: 1.6484 - categorical_accuracy: 0.3333
Epoch 00001: val_loss improved from inf to 6.60308, saving model to model_init_2021-06-2421_58_31.599331/model-00001-1.64843-0.33333-6.60308-0.23000.h5
34/34 [==============================] - 37s 1s/step - loss: 1.6484 - categorical_accuracy: 0.3333 - val_loss: 6.6031 - val_categorical_accuracy: 0.2300
Epoch 2/25
34/34 [==============================] - ETA: 0s - loss: 1.2143 - categorical_accuracy: 0.5083
Epoch 00002: val_loss improved from 6.60308 to 2.36125, saving model to model_init_2021-06-2421_58_31.599331/model-00002-1.21429-0.50830-2.36125-0.25000.h5
34/34 [==============================] - 38s 1s/step - loss: 1.2143 - categorical_accuracy: 0.5083 - val_loss: 2.3613 - val_categorical_accuracy: 0.2500
Epoch 3/25
34/34 [==============================] - ETA: 0s - loss: 0.9618 - categorical_accuracy: 0.6440
Epoch 00003: val_loss did not improve from 2.36125
34/34 [==========

In [70]:
conv_3d9=Conv3DModel5()
conv_3d9.initialize_parameters(batch_size=20,num_epochs=35,img_height=100,img_width=100)
conv_3d9_model=conv_3d9.model_structure()
conv_3d9_model.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_48 (Conv3D)           (None, 16, 98, 98, 1)     7792      
_________________________________________________________________
batch_normalization_73 (Batc (None, 16, 98, 98, 1)     4         
_________________________________________________________________
max_pooling3d_48 (MaxPooling (None, 8, 49, 49, 1)      0         
_________________________________________________________________
conv3d_49 (Conv3D)           (None, 6, 47, 47, 32)     896       
_________________________________________________________________
batch_normalization_74 (Batc (None, 6, 47, 47, 32)     128       
_________________________________________________________________
max_pooling3d_49 (MaxPooling (None, 3, 23, 23, 32)     0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 50784)           

In [71]:
conv_3d9.train_model(model=conv_3d9_model)

Epoch 1/35
34/34 [==============================] - ETA: 0s - loss: 1.7319 - categorical_accuracy: 0.3379
Epoch 00001: val_loss improved from inf to 1.85345, saving model to model_init_2021-06-2500_47_47.981610/model-00001-1.73194-0.33786-1.85345-0.17000.h5
34/34 [==============================] - 34s 992ms/step - loss: 1.7319 - categorical_accuracy: 0.3379 - val_loss: 1.8534 - val_categorical_accuracy: 0.1700
Epoch 2/35
34/34 [==============================] - ETA: 0s - loss: 1.3055 - categorical_accuracy: 0.4691
Epoch 00002: val_loss did not improve from 1.85345
34/34 [==============================] - 33s 969ms/step - loss: 1.3055 - categorical_accuracy: 0.4691 - val_loss: 2.0754 - val_categorical_accuracy: 0.2000
Epoch 3/35
34/34 [==============================] - ETA: 0s - loss: 1.0290 - categorical_accuracy: 0.6048
Epoch 00003: val_loss did not improve from 1.85345
34/34 [==============================] - 33s 968ms/step - loss: 1.0290 - categorical_accuracy: 0.6048 - val_loss: 2.